In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


In [2]:
df = pd.read_csv('dataset.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


# Feature Engineering Task 1

In [4]:
for i in df.columns[df.isnull().any(axis=0)]:
    mean = df[i].mean()
    print(f'Filling\t{df[i].isna().sum()} null value(s) in {i} with mean value -->\t{mean}')
    df[i].fillna(mean , inplace = True)

Filling	1 null value(s) in radius_mean with mean value -->	14.116125000000011
Filling	1 null value(s) in perimeter_mean with mean value -->	92.02346830985917
Filling	1 null value(s) in concavity_mean with mean value -->	0.08892480757042255
Filling	1 null value(s) in compactness_se with mean value -->	0.02546582922535212
Filling	2 null value(s) in area_worst with mean value -->	881.4024691358021
Filling	1 null value(s) in concavity_worst with mean value -->	0.27245536443661955


# SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
df = df.sample(frac = 1)
train_size=int(0.67*len(df))
X = df.drop(columns = ['diagnosis', 'id']).values
#X = np.c_[np.ones(len(X)), X]
y = df['diagnosis'].values

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
#if you do not want to shuffle, simply do not run this again

# NORMALIZING -- FEATURE ENGINEERING TASK 2

In [6]:
X_train = (X_train - X_train.mean(axis=0))/(X_train.std(axis=0))
X_test = (X_test - X_test.mean(axis=0))/(X_test.std(axis=0)) 

# IMPLEMENTING FISCHER'S LINEAR DISCRIMINANT

In [7]:
class LinearDisc:

    def _init_(self):
        self.pt = 0

    def solve(self , m1 , m2 , std1 , std2 , p1 , p2):
        a = 1/(std1**2) - 1/(std2**2)
        b = (-2*m1)/(std1**2) + (2*m2)/(std2**2)
        c = (m1**2)/(std1**2) + (m2**2)/(2*std2**2) - (2)*np.log(std1*p1/std2*p2)
        
        arr = np.roots([a,b,c])

        if(arr[0] > 0):
            print('The discriminant point is ',arr[0])
            return arr[0]
        else:
            print('The discriminant point is ',arr[1])
            return arr[1]

    def p1(x , mew , sigma):
        exp = ((-1) * (x - mew)**2)/(2 * (sigma)**2)
        val = ((np.exp ** exp)/(sigma * (2 * np.pi)**(0.5))[0])
        return val

    def find_point(self , X_train_malignant , X_train_benign , y_train):

        prior_malignant = (np.sum(y_train == 'M'))/((np.sum(y_train == 'M')) + (np.sum(y_train == 'B')))
        prior_benign = (np.sum(y_train == 'B'))/((np.sum(y_train == 'M')) + (np.sum(y_train == 'B')))

        self.pt = self.solve(np.mean(X_train_malignant) , np.mean(X_train_benign) , np.std(X_train_malignant) , np.std(X_train_benign) , prior_malignant , prior_benign)
    
    def predict(self , x):
        if(x > self.pt):
            return 'M'
        else:
            return 'B'

    def accuracy(self , y_pred , y_actual):
        acc = np.sum(y_actual == y_pred)/len(y_actual)
        return acc
    
    def metricscore(self, y_actual, y_out):
        truepos = 0;
        falsepos = 0;
        trueneg = 0;
        falseneg = 0;
        for i in range(len(y_actual)):
            if(y_actual[i] == y_out[i]):
                if(y_actual[i] == 'M'):
                    trueneg = trueneg + 1;
                else:
                    truepos = truepos + 1;
            else:
                if(y_out[i] == 'B'):
                    falseneg = falseneg + 1;
                else:
                    falsepos = falsepos + 1;
        return truepos, falsepos, trueneg, falseneg
    
    def precision(self, y_actual, y_out):
        tp, fp, tn, fn = self.metricscore(y_actual, y_out)
        prec = tp/(tp + fp)
        return prec
    
    def recall(self, y_actual, y_out):
        tp, fp, tn, fn = self.metricscore(y_actual, y_out)
        rec = tp/(tp + fn)
        return rec

In [8]:
lda = LDA(n_components=1)

X_train_1 = lda.fit_transform(X_train, y_train)
X_test_1 = lda.transform(X_test)

X_train_malignant = X_train_1[y_train == 'M']
X_train_benign = X_train_1[y_train == 'B']

ld = LinearDisc()

ld.find_point(X_train_malignant , X_train_benign , y_train)

y_predicted_train = np.array([ld.predict(i) for i in X_train_1])
y_predicted_test = np.array([ld.predict(i) for i in X_test_1])

#print('Training accuracy: ', ld.accuracy(y_predicted_train , y_train))
print('Testing accuracy: ', ld.accuracy(y_predicted_test , y_test))
#print('Training precision ',ld.precision(y_predicted_train, y_train))
print('Testing precision ',ld.precision(y_predicted_test, y_test))
#print('Training recall ', ld.recall(y_predicted_train, y_train))
print('Testing recall ',ld.recall(y_predicted_test, y_test))



The discriminant point is  0.9454598349166184
Testing accuracy:  0.8882978723404256
Testing precision  0.8372093023255814
Testing recall  1.0
